In [1]:
!pip install arrow

In [1]:
# your id
ym_app_id = 'a746048edf594f238083f56a028da76a'

In [2]:
code_request = f'https://oauth.yandex.ru/authorize?response_type=token&client_id={ym_app_id}'

In [3]:

#print("Перейти по ссылке:")
print(code_request)

https://oauth.yandex.ru/authorize?response_type=token&client_id=a746048edf594f238083f56a028da76a


In [4]:
import requests
import json
import arrow
from requests.exceptions import ConnectionError
from time import sleep


In [5]:
import sys

if sys.version_info < (3,):
    def u(x):
        try:
            return x.encode("utf8")
        except UnicodeDecodeError:
            return x
else:
    def u(x):
        if type(x) == type(b''):
            return x.decode('utf8')
        else:
            return x

In [6]:
CampaignsURL = 'https://api.direct.yandex.com/json/v5/campaigns'

In [7]:
#insert token
token = 'AgAAAAAIQ2KfAASlbI-LIUa2NE6Uvkr3rAs9_m0'
clientLogin = 'BannersA101'

In [8]:
headers = {"Authorization": "Bearer " + token,  # OAuth-токен. Использование слова Bearer обязательно
           "Client-Login": clientLogin,  # Логин клиента рекламного агентства
           "Accept-Language": "ru",  # Язык ответных сообщений
           }

In [9]:
body = {"method": "get",  # Используемый метод.
        "params": {"SelectionCriteria": {},  # Критерий отбора кампаний. Для получения всех кампаний должен быть пустым
                   "FieldNames": ["Id", "Name"]  # Имена параметров, которые требуется получить.
                   }}

In [10]:
jsonBody = json.dumps(body, ensure_ascii=False).encode('utf8')

In [11]:
try:
    result = requests.post(CampaignsURL, jsonBody, headers=headers)

    # Отладочная информация
    # print("Заголовки запроса: {}".format(result.request.headers))
    # print("Запрос: {}".format(u(result.request.body)))
    # print("Заголовки ответа: {}".format(result.headers))
    # print("Ответ: {}".format(u(result.text)))
    # print("\n")

    # Обработка запроса
    if result.status_code != 200 or result.json().get("error", False):
        print("Произошла ошибка при обращении к серверу API Директа.")
        print("Код ошибки: {}".format(result.json()["error"]["error_code"]))
        print("Описание ошибки: {}".format(u(result.json()["error"]["error_detail"])))
        print("RequestId: {}".format(result.headers.get("RequestId", False)))
    else:
        print("RequestId: {}".format(result.headers.get("RequestId", False)))
        print("Информация о баллах: {}".format(result.headers.get("Units", False)))
        # Вывод списка кампаний
        for campaign in result.json()["result"]["Campaigns"]:
            print("Рекламная кампания: {} №{}".format(u(campaign['Name']), campaign['Id']))

        if result.json()['result'].get('LimitedBy', False):
            # Если ответ содержит параметр LimitedBy, значит,  были получены не все доступные объекты.
            # В этом случае следует выполнить дополнительные запросы для получения всех объектов.
            # Подробное описание постраничной выборки - https://tech.yandex.ru/direct/doc/dg/best-practice/get-docpage/#page
            print("Получены не все доступные объекты.")


# Обработка ошибки, если не удалось соединиться с сервером API Директа
except ConnectionError:
    # В данном случае мы рекомендуем повторить запрос позднее
    print("Произошла ошибка соединения с сервером API.")

# Если возникла какая-либо другая ошибка
except:
    # В данном случае мы рекомендуем проанализировать действия приложения
    print("Произошла непредвиденная ошибка.")

Произошла ошибка при обращении к серверу API Директа.
Код ошибки: 58
Описание ошибки: Необходимо заполнить для приложения заявку на доступ в интерфейсе Директа и дождаться её подтверждения
RequestId: 8163873532122536986
